<a href="https://colab.research.google.com/github/pjablonski123/dw_matrix/blob/master/road_sign_day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [0]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [3]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks = [tensorboard_callback]
      )
  return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential((
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),
    
    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax'),
))

In [10]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 4s 16ms/step - loss: 2.5613 - accuracy: 0.3071 - val_loss: 0.9122 - val_accuracy: 0.7298
Epoch 2/5
272/272 [==============================] - 4s 15ms/step - loss: 0.6974 - accuracy: 0.7813 - val_loss: 0.1542 - val_accuracy: 0.9545
Epoch 3/5
272/272 [==============================] - 4s 15ms/step - loss: 0.3147 - accuracy: 0.9048 - val_loss: 0.0663 - val_accuracy: 0.9814
Epoch 4/5
272/272 [==============================] - 4s 15ms/step - loss: 0.2168 - accuracy: 0.9361 - val_loss: 0.0657 - val_accuracy: 0.9814
Epoch 5/5
272/272 [==============================] - 4s 15ms/step - loss: 0.1647 - accuracy: 0.9531 - val_loss: 0.0481 - val_accuracy: 0.9859


0.9605442176870749

In [11]:
model_trained.evaluate(X_test, y_test)[1]


138/138 [==============================] - 0s 4ms/step - loss: 0.1850 - accuracy: 0.9605


0.960544228553772

In [0]:
def get_model(params):
  return Sequential((
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),  
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),
    
    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax'),
))

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0,
      )
  
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [14]:
space = {
    'batch-size': hp.quniform('batch-size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch-size': 180.0, 'dropout_cnn_block_one': 0.38146698139226387, 'dropout_cnn_block_three': 0.4099028423974401, 'dropout_cnn_block_two': 0.4104286471372248, 'dropout_dense_block_one': 0.31434350439589404, 'dropout_dense_block_two': 0.6725895951349019}
accuracy=0.9126983880996704
{'batch-size': 160.0, 'dropout_cnn_block_one': 0.4008620362632538, 'dropout_cnn_block_three': 0.3150805070945904, 'dropout_cnn_block_two': 0.37499790832792645, 'dropout_dense_block_one': 0.470588631020495, 'dropout_dense_block_two': 0.6627776484653234}
accuracy=0.958730161190033
{'batch-size': 180.0, 'dropout_cnn_block_one': 0.43048324324924747, 'dropout_cnn_block_three': 0.3223433247657637, 'dropout_cnn_block_two': 0.3241466794260816, 'dropout_dense_block_one': 0.3227651421113225, 'dropout_dense_block_two': 0.41657514080336167}
accuracy=0.9498866200447083
{'batch-size': 150.0, 'dropout_cnn_block_one': 0.33397968018485946, 'dropout_cnn_block_three': 0.43794442773030956, 'dropout_cnn_block_two': 0.36957293174